<a href="https://colab.research.google.com/github/Sohom-Sarkar/AI-ML-practice/blob/main/Mnist_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb -qU
!pip install -q Cython torch
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -qU

import wandb
import torch
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor


#Verify that torch is working
print(torch.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.6/303.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.7/857.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
! pip install tqdm
from tqdm.notebook import tqdm

In [ ]:
#load data from MNIST dataset
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)


from torch.utils.data import DataLoader
#load data, shuffle , designate batch size and stuff
loaders = {
    'train': DataLoader(train_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1),

    'test': DataLoader(test_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1),
}

In [ ]:
wandb.require("core")
wandb.login(key="ce60d35946327f3382b00298b07f46a662ed5514")
wandb.init(project="Mnust_final", config={"epochs": 15})

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

for i in range(1,21):
  print("\n Run number = ",i)
  class CNN(nn.Module):

    def __init__(self):
      super(CNN, self).__init__()
      self.conv1a = nn.Conv2d(1,32, kernel_size=3)
      self.bn1a = nn.BatchNorm2d(32)
      self.conv1b = nn.Conv2d(32,32, kernel_size=3)
      self.bn1b = nn.BatchNorm2d(32)
      self.conv1c = nn.Conv2d(32,32, kernel_size=5,stride=2)
      self.conv1_drop = nn.Dropout2d(p=0.3)
      self.bn1c = nn.BatchNorm2d(32)


      self.conv2a = nn.Conv2d(32,64, kernel_size=3)
      self.bn2a = nn.BatchNorm2d(64)
      self.conv2b = nn.Conv2d(64,64, kernel_size=3)
      self.bn2b = nn.BatchNorm2d(64)
      self.conv2c = nn.Conv2d(64,64, kernel_size=5)
      self.bn2c = nn.BatchNorm2d(64)
      self.conv2_drop = nn.Dropout2d(p=0.3)

      self.fc1 = nn.Linear(256,10)


    def forward(self, x):
      x = F.relu(self.bn1a(self.conv1a(x)))
      x = F.relu(self.bn1b(self.conv1b(x)))
      x = F.relu(self.bn1c(self.conv1_drop((self.conv1c(x)))))

      x = F.relu(self.bn2a(self.conv2a(x)))
      x = F.relu(self.bn2b(self.conv2b(x)))
      x = F.relu(self.bn2c(self.conv2_drop((self.conv2c(x)))))

      x = x.view(-1,256)
      x = self.fc1(x)

      return F.log_softmax(x)


  import torch
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = CNN().to(device)

  optimizer = optim.Adam(model.parameters(), lr=0.001)
  loss_fn = nn.CrossEntropyLoss()

  def train(epoch):
      model.train()
      correct =0
      for batch_idx, (data, target) in enumerate((loaders['train'])):
          data, target = data.to(device), target.to(device)
          optimizer.zero_grad()
          output = model(data)
          loss = loss_fn(output, target)
          loss.backward()
          optimizer.step()

          pred = output.argmax(dim =1, keepdim = True)
          correct += pred.eq(target.view_as(pred)).sum().item()
          wandb.log({"loss_train": loss.item()})
          wandb.log({"accuracy_train": correct/len(loaders['train'].dataset)})
          if batch_idx %20 == 0:
              print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train"].dataset)} ({100. * batch_idx / len(loaders["train"]):.0f}%)]\t{loss.item():.6f}')

  def test(ep):
      model.eval()

      test_loss = 0
      correct = 0
      with torch.no_grad():
          for data,target in loaders['test']:
              data, target = data.to(device), target.to(device)
              output = model(data)
              test_loss += loss_fn(output,target).item()
              pred = output.argmax(dim =1, keepdim = True)
              correct += pred.eq(target.view_as(pred)).sum().item()

      test_loss /= len(loaders['test'].dataset)
      wandb.log({"accuracy_test": correct/len(loaders['test'].dataset)})
      wandb.log({"loss_test": test_loss})
      print(f'\nTest set: Average loss: {test_loss: .4f}, Accuracy {correct}/{len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):.0f}%\n')
      if(ep == 15):
        wandb.log({"Final_accuracy": correct/len(loaders['test'].dataset)})
  for epoch in tqdm(range(1,16)):
      train(epoch)
      test(epoch)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


accuracy_train,▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇██
loss_train,█▅▄▄▃▂▂▂▂▂▂▂▂▁▁▁▂▂▂▁▂▂▁▂▁▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁
accuracy_train,0.48287
loss_train,0.05038



 Run number =  1


  0%|          | 0/15 [00:00<?, ?it/s]

<ipython-input-9-4f56678843f1>:47: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	2.401393
Train Epoch: 1 [2000/60000 (3%)]	1.000449
Train Epoch: 1 [4000/60000 (7%)]	0.557380
Train Epoch: 1 [6000/60000 (10%)]	0.363021
Train Epoch: 1 [8000/60000 (13%)]	0.196953
Train Epoch: 1 [10000/60000 (17%)]	0.242091
Train Epoch: 1 [12000/60000 (20%)]	0.140311
Train Epoch: 1 [14000/60000 (23%)]	0.149559
Train Epoch: 1 [16000/60000 (27%)]	0.092994
Train Epoch: 1 [18000/60000 (30%)]	0.124781
Train Epoch: 1 [20000/60000 (33%)]	0.129323
Train Epoch: 1 [22000/60000 (37%)]	0.155680
Train Epoch: 1 [24000/60000 (40%)]	0.062519
Train Epoch: 1 [26000/60000 (43%)]	0.131774
Train Epoch: 1 [28000/60000 (47%)]	0.091228
Train Epoch: 1 [30000/60000 (50%)]	0.141484
Train Epoch: 1 [32000/60000 (53%)]	0.205007
Train Epoch: 1 [34000/60000 (57%)]	0.045268
Train Epoch: 1 [36000/60000 (60%)]	0.031384
Train Epoch: 1 [38000/60000 (63%)]	0.059535
Train Epoch: 1 [40000/60000 (67%)]	0.123084
Train Epoch: 1 [42000/60000 (70%)]	0.088823
Train Epoch: 1 [44000/60000 (73%)]	0.077612